In [1]:
%%sh
pygmentize train_function.R

library("rjson")

train <- function() {
    hp <- fromJSON(file='/opt/ml/input/config/hyperparameters.json')
    print(hp)
    normalize <- hp$normalize

    data <- read.csv(file='/opt/ml/input/data/training/housing.csv', header=T)
    if (normalize) {
        data <- as.data.frame(scale(data))
    }
    print(summary(data))

    model = lm(medv~., data)
    print(summary(model))

    saveRDS(model, '/opt/ml/model/model.rds')
}


In [2]:
%%sh
pygmentize serve_function.R

#' @get /ping
function() {
    return('')}

#' @param req The http request sent
#' @post /invocations
function(req) {    
    model <- readRDS('/opt/ml/model/model.rds')
    conn <- textConnection(gsub('\\\\n', '\n', req$postBody))
    data <- read.csv(conn)
    close(conn)
    print(data)
    medv <- predict(model, data)
    return(medv)
}


In [3]:
%%sh
pygmentize main.R

library('plumber')

source('train_function.R')

serve <- function() {
    app <- plumb('serve_function.R')
    app$run(host='0.0.0.0', port=8080)}

args <- commandArgs()
if (any(grepl('train', args))) {
    train()
}
if (any(grepl('serve', args))) {
    serve()
}


In [4]:
%%sh
cat Dockerfile

FROM r-base:latest

RUN R -e "install.packages(c('rjson', 'plumber'), repos='https://cloud.r-project.org')"

COPY main.R /opt/ml/
COPY train_function.R /opt/ml/
COPY serve_function.R /opt/ml/

WORKDIR /opt/ml/

ENTRYPOINT ["/usr/bin/Rscript", "/opt/ml/main.R", "--no-save"]

In [5]:
%%sh
aws ecr create-repository --repository-name r-custom


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'r-custom' already exists in the registry with id '613904931467'


In [6]:
%%sh
export REGION=eu-west-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t r-custom:predictor -f Dockerfile .
export IMAGE_ID=`docker images -q r-custom:predictor`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:predictor
aws ecr get-login-password | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:predictor
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:predictor


Step 1/7 : FROM r-base:latest
latest: Pulling from library/r-base
73b408cb748b: Pulling fs layer
5216627df1d4: Pulling fs layer
5233fa73a943: Pulling fs layer
224d21aef6e2: Pulling fs layer
d3491ee7b75d: Pulling fs layer
5fd8bc235aa1: Pulling fs layer
224d21aef6e2: Waiting
5fd8bc235aa1: Waiting
d3491ee7b75d: Waiting
5216627df1d4: Verifying Checksum
5216627df1d4: Download complete
5233fa73a943: Download complete
224d21aef6e2: Download complete
73b408cb748b: Verifying Checksum
73b408cb748b: Download complete
d3491ee7b75d: Verifying Checksum
d3491ee7b75d: Download complete
73b408cb748b: Pull complete
5216627df1d4: Pull complete
5233fa73a943: Pull complete
224d21aef6e2: Pull complete
d3491ee7b75d: Pull complete
5fd8bc235aa1: Verifying Checksum
5fd8bc235aa1: Download complete
5fd8bc235aa1: Pull complete
Digest: sha256:e292b19eb361d5494ee5eab580db3dc11f1ecb59200c1bc1625f81e78a67410b
Status: Downloaded newer image for r-base:latest
 ---> d984908c70c0
Step 2/7 : RUN R -e "install.packages(c('

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [7]:
import sagemaker
from sagemaker.estimator import Estimator

print(sagemaker.__version__)

session = sagemaker.Session()
account_id = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name

training = 'file://.'
output = 'file://.'

role = sagemaker.get_execution_role()
r_estimator = Estimator(
             image_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/r-custom:predictor',
             role=role,
             instance_count=1, 
             instance_type='local',
             output_path=output,
             hyperparameters={
                  'normalize': False
             }
)

r_estimator.fit({'training':training})

2.0.0rc1


TypeError: __init__() got an unexpected keyword argument 'image_name'

In [ ]:
%%sh
tar tvfz model.tar.gz

In [ ]:
r_predictor = r_estimator.deploy(initial_instance_count=1, instance_type='local')

In [ ]:
import pandas as pd

data = pd.read_csv('housing.csv')
data.drop(['medv'], axis=1, inplace=True)
data = data.to_csv(index=False)

r_predictor.content_type = 'text/csv'
r_predictor.serializer = sagemaker.serializers.CSVSerializer()

response = r_predictor.predict(data)

In [ ]:
r_predictor.delete_endpoint()